# Finding putative metabolite interactions

We will employ [SMETANA]() through its [Python implementation]() to find putative metabolic interactions among the set of models in our use case: a phycosphere microbiome composed of the diatom: Pseudo-nitzschia sp., and the bacterial partners: Sulfitobacter sp., Alteromonas sp., Marinobacter sp., and Polaribacter sp.

To this end, we will simulate a marine medium containing only inorganic componets, as vitamins and cofactors will be synthesized by the bacterial partners, while the diatom will provide the carbon source.

## Which bacterial partners are producing Pseudo-nitzschia essential cofactors?

* Adenosylcobalamin (adocbl): _Sulfitobacter sp._
* Folate (fol): _?_
* Thiamin (thm): _?_ https://www.frontiersin.org/articles/10.3389/fmars.2020.606342/full


It seems like none of the reconstructed GEMs are producing thiamin and folate per se, but rather importing. Is this a limitation of the universal model or are these species really auxotrophs for them?

| Model | B12 (adocbl) | B1 (thm) | Folate |
|-------|----------|----------|--------|
| Sulfitobacter   | +   | -   | -   |
| Alteromonas   | -   | -   | -   |
| Marinobacter   | -   | -   | -   |
| Polaribacter   | -   | -   | -   |

In [35]:
%%bash

smetana \
    ../results/models/*.xml \
    -m MARINE \
    --mediadb ../data/marine_media/media_db.tsv \
    -o ../results/pseudo_nitzschia_interactions \
    --solver gurobi \
    --exclude ../data/compounds/inorganic.txt \
    --molweight --detailed

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


/home/robaina/.local/lib/python3.8/site-packages/reframed/core/elements.py:62: UserWarning: Atomic weight not listed for elements: {'Z'}
  warn(f"Atomic weight not listed for elements: {missing}")


In [37]:
import pandas as pd


df = pd.read_csv("../results/pseudo_nitzschia_interactions_detailed.tsv", sep="\t")
df

,community,medium,receiver,donor,compound,scs,mus,mps,smetana


## Which compounds can PS secrete?

In [41]:
import cobra 


model = cobra.io.read_sbml_model("../results/models/MAG_00212_pseudonitzschia_photoeuk_noSK.xml")

In [5]:
model.reactions.get_by_id("EX_tsul_e")
model.metabolites.get_by_id("CPD-15199_c")

Metabolite identifier,CPD-15199_c
Name,5-deoxy-α-ribose 1-phosphate
Memory address,0x7fc6c0ec93d0
Formula,C5H9O7P
Compartment,c
In 2 reaction(s),"RXN-14304, DM_CPD-15199_c"


In [42]:
import re 
from cobra.core import Model


def extract_chemical_elements(formula: str) -> dict:
    """
    Extract the chemical components from a chemical formula.
    """
    components = re.findall(r"([A-Z][a-z]*)(\d*)", formula)
    component_dict = {}
    for element, count in components:
        component_dict[element] = int(count) if count else 1
    return component_dict


def get_organic_exchanges(model: Model, exclude: list = None) -> list[str]:
    """
    Get IDs of all organic exchanges in a model.

    Parameters
    ----------
    model : cobra.Model
        Model to get exchanges from.

    Returns
    -------
    list
        List of exchange IDs.
    """
    organic_exchanges = []
    for rxn in model.exchanges:
        met = [met for met in rxn.metabolites][0]
        if met.formula is not None:
            chemical_elements = extract_chemical_elements(met.formula)
            if "C" in chemical_elements:
                organic_exchanges.append(rxn.id)
    return organic_exchanges


carbon_exchanges = get_organic_exchanges(model)

In [45]:
from cobra.flux_analysis.variability import flux_variability_analysis, find_blocked_reactions



fva_ex = flux_variability_analysis(model, reaction_list=carbon_exchanges, loopless=False, fraction_of_optimum=0.99)

In [46]:
fva_ex[fva_ex.maximum > 0]

,minimum,maximum
EX_glyclt_e,0.0,0.829982
EX_glc_e,0.0,0.207495
EX_tcynt_e,0.0,0.355706
EX_lac_d_e,0.0,0.414991
EX_etoh_e,0.0,0.829982
...,...,...
EX_ser_L_e,0.0,0.414991
EX_13dpg_e,0.0,0.414991
EX_2aobut_e,0.0,0.414991
EX_ala_B_e,0.0,0.355706
